# Math Tutor Assistant

## Objective

This notebook demonstrates the following:

- Showcases the foundational concepts of Assistants such as Threads, Messages, Runs, Tools, and lifecycle management.

This tutorial uses the following Azure AI services:

- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)\n",
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:

- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant).
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)
- [Github-OpenAI Python/examples/Assistant demo notebook](https://github.com/openai/openai-python/blob/main/examples/assistant.py)

## Time

You should expect to spend 5-10 minutes running this sample. 

## About this example

This sample shows users how to create an Azure OpenAI Assistant named "Math Tutor" using the Azure OpenAI API. The assistant is designed to function as a personal math tutor, capable of answering math questions through code interpretation. The script initiates a conversation with the assistant, guiding it through various mathematical queries and scenarios to showcase its capabilities.

This sample provides developers with a clear demonstration of how to leverage the core concepts of the Assistants API into their projects, highlighting its simplicity and effectiveness in leveraging foundational concepts. 

## Load the required libraries

In [ ]:
#r "nuget: Azure.AI.OpenAI.Assistants, 1.0.0-beta.3"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI.Assistants;

## Load the environment variables

In [ ]:
using dotenv.net;
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] {"../.env"}));

var api_endpoint = Environment.GetEnvironmentVariable("OPENAI_URI");
var api_key = Environment.GetEnvironmentVariable("OPENAI_KEY");
var api_version = Environment.GetEnvironmentVariable("OPENAI_VERSION");
var api_deployment_name = Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");
HTML(api_deployment_name)


### Create an Azure OpenAI client

In [ ]:
AssistantsClient client = new AssistantsClient(new Uri(api_endpoint), new AzureKeyCredential(api_key));

In [ ]:
Response<Assistant> assistantResponse = await client.CreateAssistantAsync(
    new AssistantCreationOptions(api_deployment_name)
    {
        Name = "Math Tutor",
        Instructions = "You are a personal math tutor. Write and run code to answer math questions.",
        Tools = { new CodeInterpreterToolDefinition() }        
    });
Assistant assistant = assistantResponse.Value;

### Create an Assistant and a Thread

In [ ]:
Response<AssistantThread> threadResponse = await client.CreateThreadAsync();
AssistantThread thread = threadResponse.Value;

In [ ]:
async Task ProcessMessagesAsync(IReadOnlyList<ThreadMessage> messages)
{
    List<ThreadMessage> localList = [];
    foreach (ThreadMessage threadMessage in messages)
    {
        localList.Add(threadMessage);
        if (threadMessage.Role == "user")
        {
            break;
        }
    }
    localList.Reverse();

    // Note: messages iterate from newest to oldest, with the messages[0] being the most recent
    foreach (ThreadMessage threadMessage in localList)
    {
        Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
        foreach (MessageContent contentItem in threadMessage.ContentItems)
        {
            if (contentItem is MessageTextContent textItem)
            {
                Console.Write(textItem.Text);
                var annotations = textItem.Annotations;
                if (annotations != null && annotations.Count > 0)
                {
                    foreach (var annotation in annotations)
                    {
                        if (annotation is MessageTextFileCitationAnnotation textCitation)
                        {
                            Console.Write(textCitation.Text);
                        } else if (annotation is MessageTextFilePathAnnotation filCitation)
                        {
                            Console.Write(filCitation.FileId);
                        }
                    }
                }
                
            }
            else if (contentItem is MessageImageFileContent imageFileItem)
            {
                // Console.Write($"<image from ID: {imageFileItem.FileId}");
                var resp = await client.GetFileContentAsync(imageFileItem.FileId);
                if (resp.Value is not null)
                {
                    var b64 = Convert.ToBase64String(resp.Value);
                    var img = $"<img src='data:image/png;base64,{b64}' />";
                    HTML(img);
                }
            }
            Console.WriteLine();
        }
    }
}

In [ ]:
async Task ProcessPrompt(string prompt)
{
    Response<ThreadMessage> messageResponse = await client.CreateMessageAsync(
        thread.Id,
        MessageRole.User,
        prompt);
    ThreadMessage message = messageResponse.Value;
    
    Response<ThreadRun> runResponse = await client.CreateRunAsync(thread,assistant);
    ThreadRun run = runResponse.Value;

    do
    {
        await Task.Delay(TimeSpan.FromMilliseconds(500));
        runResponse = await client.GetRunAsync(thread.Id, runResponse.Value.Id);
        if (runResponse.Value.Status == RunStatus.RequiresAction)
        {
            // Process Funcion calling
        }
    }
    while (runResponse.Value.Status == RunStatus.Queued
    || runResponse.Value.Status == RunStatus.InProgress);

    Response<PageableList<ThreadMessage>> afterRunMessagesResponse
    = await client.GetMessagesAsync(thread.Id);
    IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

    await ProcessMessagesAsync(messages);
}


### Have a conversation with the Assistant

In [ ]:
await ProcessPrompt("What is the linear equation when two (x,y) points are (1,1) and (5,10)?");

In [ ]:
await ProcessPrompt("I need to solve the equation `3x + 11 = 14`. Can you help me?");

In [ ]:
await ProcessPrompt(@"x=r*cos(u)sin(v), y=r*sin(u)sin(v), r=2+sin(7*u+5*v) for 0<u<2π and 0<v<π.
Create a graph of the equation z=r*cos(v).");

In [ ]:
await ProcessPrompt("create a csv file with 10 customer names")

### Cleanup

In [ ]:
await client.DeleteAssistantAsync(assistant.Id);
await client.DeleteThreadAsync(thread.Id);